In [173]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Conv2D, Flatten
from keras.optimizers import SGD, rmsprop
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical

In [174]:
num_classes = 10

def get_data(train_size=49000, validation_size=1000, test_size=10000):
    # The data, shuffled and split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    mean_image = np.mean(x_train, axis=0)
    x_train = (x_train.astype('float64') - mean_image)
    x_test = (x_test.astype('float64') - mean_image)

    # x_train = x_train.reshape(x_train.shape[0], -1)
    # x_test = x_test.reshape(x_test.shape[0], -1)

    x_val = x_train[train_size:train_size+validation_size]
    y_val = y_train[train_size:train_size+validation_size]
    x_train = x_train[:train_size]
    y_train = y_train[:train_size]
    x_test = x_test[:test_size]
    y_test = y_test[:test_size]
    return x_train, y_train, x_val, y_val, x_test, y_test

In [143]:
# Model: Dense - ReLu - Dense - Softmax
x_train, y_train, x_val, y_val, x_test, y_test = get_data(49000, 1000, 10000)

results = {}
for hidden in [120]:
    for reg in [1.0]:
        for lr in [7e-4]:
            for decay in [8e-3]:
                model = Sequential([
                    Dense(hidden, input_dim=3 * 32 * 32, kernel_regularizer=l2(reg)),
                    Activation('relu'),
                    Dense(10, kernel_regularizer=l2(reg)),
                    Activation('softmax')
                ])
                model.compile(
                    optimizer=SGD(lr=lr, decay=decay, momentum=0.9, nesterov=True),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
                model.fit(
                    x_train, to_categorical(y_train, num_classes),
                    epochs=10, batch_size=200, shuffle=True, verbose=False
                )
                preds = np.argmax(model.predict(x_test), axis=1)
                accuracy = np.mean(y_test.reshape(-1) == preds)
                print('{} {} {} {}: {}'.format(hidden, reg, lr, decay, accuracy))
                results[(hidden, reg, lr, decay)] = accuracy

ValueError: Error when checking model input: expected dense_851_input to have 2 dimensions, but got array with shape (49000, 32, 32, 3)

In [175]:
x_train, y_train, x_val, y_val, x_test, y_test = get_data(10000, 1000, 10000)
x_test, y_test = x_val, y_val

model = Sequential([
    Conv2D(input_shape=x_train.shape[1:], filters=32, kernel_size=(3,3), padding='same'),
    Activation('relu'),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(10),
    Activation('softmax')
])
model.compile(
    optimizer=rmsprop(lr=1e-4, decay=1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(
    x_train, to_categorical(y_train, num_classes),
    epochs=100, batch_size=200, shuffle=True, verbose=True
)
preds = np.argmax(model.predict(x_test), axis=1)
accuracy = np.mean(y_test.reshape(-1) == preds)
print(accuracy)

Epoch 1/100
10000/10000 [==============================] - 42s - loss: 2.5623 - acc: 0.1858      